In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

c:\python27\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [23]:
#读入数据()
paidan_data = pd.read_csv('paidan_data/paidan_model_data_nanjing.csv', index_col = 1)
paidan_data = paidan_data.fillna(value = 0)


#异常值处理
paidan_data.loc[paidan_data.paidan_receiver_distance_jiedan_supplier_max > 10000,"paidan_receiver_distance_jiedan_supplier_max"] = paidan_data["paidan_receiver_distance_jiedan_supplier_max"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.paidan_receiver_distance_jiedan_supplier_min > 10000,"paidan_receiver_distance_jiedan_supplier_min"] = paidan_data["paidan_receiver_distance_jiedan_supplier_min"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.paidan_supplier_distance_jiedan_supplier_max > 10000,"paidan_supplier_distance_jiedan_supplier_max"] = paidan_data["paidan_supplier_distance_jiedan_supplier_max"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.paidan_supplier_distance_jiedan_supplier_min > 10000,"paidan_supplier_distance_jiedan_supplier_min"] = paidan_data["paidan_supplier_distance_jiedan_supplier_min"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.paidan_receiver_distance_jiedan_receiver_max > 10000,"paidan_receiver_distance_jiedan_receiver_max"] = paidan_data["paidan_receiver_distance_jiedan_receiver_max"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.paidan_receiver_distance_jiedan_receiver_min > 10000,"paidan_receiver_distance_jiedan_receiver_min"] = paidan_data["paidan_receiver_distance_jiedan_receiver_min"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.paidan_supplier_distance_jiedan_receiver_max > 10000,"paidan_supplier_distance_jiedan_receiver_max"] = paidan_data["paidan_supplier_distance_jiedan_receiver_max"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.paidan_supplier_distance_jiedan_receiver_min > 10000,"paidan_supplier_distance_jiedan_receiver_min"] = paidan_data["paidan_supplier_distance_jiedan_receiver_min"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.supplier_receiver_distance > 10000,"supplier_receiver_distance"] = paidan_data["supplier_receiver_distance"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan_data.loc[paidan_data.transporter_supplier_distance > 10000,"transporter_supplier_distance"] = paidan_data["transporter_supplier_distance"][paidan_data.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()


# varl_ten = [
#         'paidan_hour',
#         'allowance_amt',   
#         'transporter_before_seven_day_paidan_lv',          
#         'transporter_before_seven_day_paidan_success_num',
#         'transporter_before_seven_day_jiedan_success_num',
#         'transporter_before_fourteen_day_jiedan_success_num',
#         'transporter_before_fourteen_day_paidan_lv',
#         'today_finish_order_num',
#         'today_never_finish_order_num',
#         'supplier_receiver_distance',
#         'transporter_supplier_distance',
#         'paidan_receiver_distance_jiedan_receiver_min',
#         'paidan_supplier_distance_jiedan_receiver_min',
#         'paidan_supplier_distance_jiedan_receiver_max',
#         'paidan_receiver_distance_jiedan_receiver_max',
#         'paidan_supplier_distance_jiedan_supplier_min'
# ]


varl_ten = ['supplier_receiver_distance',
        'transporter_supplier_distance',
        'today_finish_order_num',
        'transporter_before_fourteen_day_paidan_lv',
        'paidan_receiver_distance_jiedan_receiver_min',
        'paidan_supplier_distance_jiedan_receiver_min',
        'paidan_hour',
        'transporter_before_seven_day_paidan_lv',
        'paidan_supplier_distance_jiedan_receiver_max',
        'paidan_receiver_distance_jiedan_receiver_max',
        'allowance_amt']





#区分目标变量和
y_train = paidan_data.iloc[:,-1]
x_train = paidan_data.ix[:,varl_ten]


#最大最小归一化
x_train_columns = x_train.columns
x_train_index = x_train.index
min_max_scaler = preprocessing.MinMaxScaler()
x_train = min_max_scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns = x_train_columns, index = x_train_index)



In [25]:
#读入数据
paidan = pd.read_csv('paidan_data/paidan_model_test_data_1214.csv', index_col = 1)
paidan = paidan.fillna(value = 0)

#异常值处理
paidan.loc[paidan.paidan_receiver_distance_jiedan_supplier_max > 10000,"paidan_receiver_distance_jiedan_supplier_max"] = paidan["paidan_receiver_distance_jiedan_supplier_max"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.paidan_receiver_distance_jiedan_supplier_min > 10000,"paidan_receiver_distance_jiedan_supplier_min"] = paidan["paidan_receiver_distance_jiedan_supplier_min"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.paidan_supplier_distance_jiedan_supplier_max > 10000,"paidan_supplier_distance_jiedan_supplier_max"] = paidan["paidan_supplier_distance_jiedan_supplier_max"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.paidan_supplier_distance_jiedan_supplier_min > 10000,"paidan_supplier_distance_jiedan_supplier_min"] = paidan["paidan_supplier_distance_jiedan_supplier_min"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.paidan_receiver_distance_jiedan_receiver_max > 10000,"paidan_receiver_distance_jiedan_receiver_max"] = paidan["paidan_receiver_distance_jiedan_receiver_max"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.paidan_receiver_distance_jiedan_receiver_min > 10000,"paidan_receiver_distance_jiedan_receiver_min"] = paidan["paidan_receiver_distance_jiedan_receiver_min"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.paidan_supplier_distance_jiedan_receiver_max > 10000,"paidan_supplier_distance_jiedan_receiver_max"] = paidan["paidan_supplier_distance_jiedan_receiver_max"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.paidan_supplier_distance_jiedan_receiver_min > 10000,"paidan_supplier_distance_jiedan_receiver_min"] = paidan["paidan_supplier_distance_jiedan_receiver_min"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.supplier_receiver_distance > 10000,"supplier_receiver_distance"] = paidan["supplier_receiver_distance"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()
paidan.loc[paidan.transporter_supplier_distance > 10000,"transporter_supplier_distance"] = paidan["transporter_supplier_distance"][paidan.paidan_receiver_distance_jiedan_supplier_max < 10000].mean()


y_paidan = paidan.ix[:,-1]
x_paidan = paidan.ix[:,varl_ten]

x_paidan_index = x_paidan.index
x_paidan_columns = x_paidan.columns
x_paidan = min_max_scaler.transform(x_paidan)
x_paidan =pd.DataFrame(x_paidan, columns = x_paidan_columns, index = x_paidan_index)

In [26]:
clf_regression = LogisticRegression(C = 100)
clf_regression.fit(x_train,y_train)

test_label = clf_regression.predict(x_paidan)

a1 = accuracy_score(y_paidan,test_label)
a = f1_score(y_paidan,test_label)
b = precision_score(y_paidan,test_label)
c = recall_score(y_paidan,test_label)
d = roc_auc_score(y_paidan,test_label)
e = confusion_matrix(y_paidan,test_label)

print '准确率:',a1
print 'F1:',a
print '命中率：',b
print '查全率：',c
print 'auc:',d
print '混淆矩阵：'
print e

准确率: 0.745776347546
F1: 0.534472598704
命中率： 0.647702927874
查全率： 0.454940645377
auc: 0.669036968298
混淆矩阵：
[[11184  1480]
 [ 3260  2721]]


In [7]:
w = clf_regression.coef_

(array([[ -8.00172346e-04,  -2.16890280e-04,   1.90038772e-02,
           1.71914989e+00,  -7.11172686e-04,   1.00583505e-03,
          -7.29643117e-04,   1.60844598e+00,   2.10186878e-04,
          -1.96999771e-04,   1.51737581e-03]]),

array([[ -7.51690035e-04,  -4.93958593e-04,   1.81555386e-02,
          1.75241148e+00,  -4.61868289e-04,   9.35225491e-04,
          5.55834868e-03,   1.71082425e+00,   2.59188044e-04,
         -1.81712059e-04,   3.01172246e-03]])

In [9]:
b = clf_regression.intercept_ 
b

array([-1.41401794])

In [20]:
test_paidan = x_paidan.copy()
test_paidan['label'] = y_paidan
test_paidan['pre_label'] = test_label
jishu = np.dot(x_paidan,w.reshape(11,1)) + b
test_paidan['jishu'] = jishu
test_paidan.to_csv('c:/test_paidan.csv')
test_paidan

,supplier_receiver_distance,transporter_supplier_distance,today_finish_order_num,transporter_before_fourteen_day_paidan_lv,paidan_receiver_distance_jiedan_receiver_min,paidan_supplier_distance_jiedan_receiver_min,paidan_hour,transporter_before_seven_day_paidan_lv,paidan_supplier_distance_jiedan_receiver_max,paidan_receiver_distance_jiedan_receiver_max,allowance_amt,label,pre_label,jishu
order_supplier_transporter,,,,,,,,,,,,,,
125738223_-99_75322_118305,378.656596,49.687311,0,0.10559,0.000000,0.000000,0,0.14130,0.000000,0.000000,3,-1,-1,-1.287382
125738223_-99_75322_652303,378.656596,55.417692,0,0.45425,0.000000,0.000000,0,0.42675,0.000000,0.000000,3,-1,-1,-0.190862
125738455_-99_100782_19429,1116.295644,5.534682,0,0.11429,0.000000,0.000000,0,0.13115,0.000000,0.000000,4,-1,-1,-1.819156
125738459_-99_10022_12810,1341.737186,16.533386,0,0.00000,0.000000,0.000000,0,0.00000,0.000000,0.000000,0,-1,-1,-2.430755
125738455_-99_100782_312992,1116.295644,82.629371,0,0.11409,0.000000,0.000000,0,0.11268,0.000000,0.000000,4,-1,-1,-1.889187
125738561_-99_102088_432257,1014.995247,20.820313,0,0.88462,0.000000,0.000000,0,1.00000,0.000000,0.000000,3,1,1,1.082814
125738601_-99_98453_283886,1480.224355,84.264869,0,0.43636,0.000000,0.000000,0,0.35000,0.000000,0.000000,3,-1,-1,-1.195805
125738636_-99_10022_12810,1209.180981,16.533386,0,0.00000,0.000000,0.000000,0,0.00000,0.000000,0.000000,3,-1,-1,-2.322079
125738639_-99_102088_432257,619.885302,41.369288,0,0.88462,420.933613,1014.995247,0,1.00000,1014.995247,420.933613,3,1,1,2.311083


In [27]:
paidan = pd.read_csv('paidan_data/paidan_model_test_data_two_day.csv')
paidan = paidan.fillna(value = 0)

#异常值处理
paidan.loc[paidan.paidan_receiver_distance_jiedan_receiver_max > 10000,"paidan_receiver_distance_jiedan_receiver_max"] = paidan["paidan_receiver_distance_jiedan_receiver_max"][paidan.paidan_receiver_distance_jiedan_receiver_max < 10000].mean()
paidan.loc[paidan.paidan_receiver_distance_jiedan_receiver_min > 10000,"paidan_receiver_distance_jiedan_receiver_min"] = paidan["paidan_receiver_distance_jiedan_receiver_min"][paidan.paidan_receiver_distance_jiedan_receiver_min < 10000].mean()
paidan.loc[paidan.paidan_supplier_distance_jiedan_receiver_max > 10000,"paidan_supplier_distance_jiedan_receiver_max"] = paidan["paidan_supplier_distance_jiedan_receiver_max"][paidan.paidan_supplier_distance_jiedan_receiver_max < 10000].mean()
paidan.loc[paidan.paidan_supplier_distance_jiedan_receiver_min > 10000,"paidan_supplier_distance_jiedan_receiver_min"] = paidan["paidan_supplier_distance_jiedan_receiver_min"][paidan.paidan_supplier_distance_jiedan_receiver_min < 10000].mean()
paidan.loc[paidan.supplier_receiver_distance > 10000,"supplier_receiver_distance"] = paidan["supplier_receiver_distance"][paidan.supplier_receiver_distance < 10000].mean()
paidan.loc[paidan.transporter_supplier_distance > 10000,"transporter_supplier_distance"] = paidan["transporter_supplier_distance"][paidan.transporter_supplier_distance < 10000].mean()


y_paidan = paidan.ix[:,-1]
x_paidan = paidan.ix[:,:-1]
x_paidan_index = x_paidan.index
x_paidan_columns = x_paidan.columns
x_paidan = min_max_scaler.transform(x_paidan)
x_paidan =pd.DataFrame(x_paidan, columns = x_paidan_columns, index = x_paidan_index)

In [28]:
test_label = clf_regression.predict(x_paidan)

a1 = accuracy_score(y_paidan,test_label)
a = f1_score(y_paidan,test_label)
b = precision_score(y_paidan,test_label)
c = recall_score(y_paidan,test_label)
d = roc_auc_score(y_paidan,test_label)
e = confusion_matrix(y_paidan,test_label)

print '准确率:',a1
print 'F1:',a
print '命中率：',b
print '查全率：',c
print 'auc:',d
print '混淆矩阵：'
print e

准确率: 0.747376145157
F1: 0.560111511209
命中率： 0.627864583333
查全率： 0.505556720486
auc: 0.682877822171
混淆矩阵：
[[26378  4287]
 [ 7074  7233]]


In [33]:
w = np.array([[ -8.00172346e-04,  -2.16890280e-04,   1.90038772e-02,
           1.71914989e+00,  -7.11172686e-04,   1.00583505e-03,
          -7.29643117e-04,   1.60844598e+00,   2.10186878e-04,
          -1.96999771e-04,   1.51737581e-03]])
b = np.array([-1.25077483])
w,b


(array([[ -8.00172346e-04,  -2.16890280e-04,   1.90038772e-02,
           1.71914989e+00,  -7.11172686e-04,   1.00583505e-03,
          -7.29643117e-04,   1.60844598e+00,   2.10186878e-04,
          -1.96999771e-04,   1.51737581e-03]]), array([-1.25077483]))

In [37]:
w = pd.DataFrame(w)
b = pd.DataFrame(b)
w.to_csv('c:/w.csv')
b.to_csv('c:/d.csv')